In [6]:
import torch

class Attention(torch.nn.Module):
    def __init__(self, dim: int) -> None:
        super().__init__()
        self.wq = torch.nn.Linear(dim, dim, bias=False)
        self.wk = torch.nn.Linear(dim, dim, bias=False)
        self.wv = torch.nn.Linear(dim, dim, bias=False)

    def forward(self, x: torch.Tensor, mask: torch.Tensor | None = None) -> torch.Tensor:
        Q = self.wq(x)  # [batch_size, seq_len, dim]
        K = self.wk(x)  # [batch_size, seq_len, dim]
        V = self.wv(x)  # [batch_size, seq_len, dim]

        # Compute scaled dot-product attention
        attention = Q @ K.transpose(-2, -1)  # [batch_size, seq_len, seq_len]
        attention /= Q.shape[-1] ** 0.5  # Scale by sqrt of dimension
        
        if mask is not None:  # Check for mask presence
            attention += mask.to(attention.dtype) * -1e9
        
        attention = torch.softmax(attention, dim=-1)  # Apply softmax along seq_len

        # Weighted sum of values
        output = attention @ V  # [batch_size, seq_len, dim]
        return output


# Test the updated Attention module

# Create a random input tensor
batch_size = 2
seq_len = 5
dim = 8
x = torch.randn(batch_size, seq_len, dim)

# Instantiate the Attention module
attention_module = Attention(dim=dim)

# Test 1: Without Mask
output_no_mask = attention_module(x)

# Test 2: With Mask (causal mask example)
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).unsqueeze(0).repeat(batch_size, 1, 1) * -1e9
output_with_mask = attention_module(x, mask=mask)

# Output shapes
output_shapes = {
    "without_mask": output_no_mask.shape,
    "with_mask": output_with_mask.shape
}
output_shapes


{'without_mask': torch.Size([2, 5, 8]), 'with_mask': torch.Size([2, 5, 8])}